#### ATMS 523 Assignment 5

Haley Schmidt

10/31/25

In [31]:
# Setup
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.ensemble import RandomForestRegressor


In [32]:
# Load data
data = pd.read_csv("homework/radar_parameters.csv").rename(columns={
    "Zh (dBZ)": "Zh",
    "Zdr (dB)": "Zdr",
    "Ldr (dB)": "Ldr",
    "Kdp (deg km-1)": "Kdp",
    "Ah (dBZ/km)": "Ah",
    "Adr (dB/km)": "Adr",
    "R (mm/hr)": "R"
})
data

,Unnamed: 0,Zh,Zdr,Ldr,Kdp,Ah,Adr,R
0,0,23.144878,0.418637,-41.757733,0.005395,0.000290,0.000012,2.393520
1,1,22.737156,0.322850,-43.772069,0.005194,0.000360,0.000012,3.502699
2,2,26.869826,0.330948,-43.577399,0.013385,0.000903,0.000030,8.627561
3,3,28.540561,0.399480,-42.139731,0.018872,0.001036,0.000043,8.424447
4,4,30.500127,0.543758,-39.763087,0.027438,0.001157,0.000064,8.189291
...,...,...,...,...,...,...,...,...
18964,18964,31.515997,0.579955,-39.244229,0.034048,0.001417,0.000080,10.648020
18965,18965,29.993334,0.567935,-39.399188,0.024134,0.001032,0.000057,7.981875
18966,18966,31.685913,0.655681,-38.375696,0.033971,0.001165,0.000081,6.822691
18967,18967,32.980096,0.768586,-37.166218,0.043117,0.001285,0.000105,6.801169


1. Split the data into a 70-30 split for training and testing data.

In [33]:
# Separate features and target
X = data[["Zh", "Zdr", "Ldr", "Kdp", "Ah", "Adr"]]  
y = data["R"]

# Split data 70-30
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42
    )

2. Using the split created in (1), train a multiple linear regression dataset using the training dataset, and validate it using the testing dataset.  Compare the $R^2$ and root mean square errors of model on the training and testing sets to a baseline prediction of rain rate using the formula $Z = 200 R^{1.6}$.

In [34]:
# Train model
model = LinearRegression()
model.fit(X_train, y_train)

,fit_intercept,True
,copy_X,True
,tol,1e-06
,n_jobs,None
,positive,False


In [35]:
# predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

In [36]:
# performance eval
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"R^2 (Train): {r2_train:.3f}")
print(f"R^2 (Test): {r2_test:.3f}")
print(f"RMSE (Train): {rmse_train:.3f}")
print(f"RMSE (Test): {rmse_test:.3f}")

R^2 (Train): 0.988
R^2 (Test): 0.989
RMSE (Train): 0.923
RMSE (Test): 0.936


In [37]:
# Baseline model - Z = 200 * R^1.6 

Z_lin = 10 ** (data["Zh"] / 10)   # convert dBZ to Z
R_baseline = (Z_lin / 200) ** (1 / 1.6)

# baseline R^2 and RMSE 
R_baseline_train = (10 ** (X_train["Zh"] / 10) / 200) ** (1 / 1.6)
R_baseline_test = (10 ** (X_test["Zh"] / 10) / 200) ** (1 / 1.6)

# performance eval
r2_train_base = r2_score(y_train, R_baseline_train)
r2_test_base = r2_score(y_test, R_baseline_test)
rmse_train_base = np.sqrt(mean_squared_error(y_train, R_baseline_train))
rmse_test_base = np.sqrt(mean_squared_error(y_test, R_baseline_test))

print(f"R^2 (Train): {r2_train_base:.3f}")
print(f"R^2 (Test): {r2_test_base:.3f}")
print(f"RMSE (Train): {rmse_train_base:.3f}")
print(f"RMSE (Test): {rmse_test_base:.3f}")

R^2 (Train): 0.276
R^2 (Test): 0.357
RMSE (Train): 7.144
RMSE (Test): 7.189


3. Repeat 1 doing a grid search over polynomial orders, using a grid search over orders 0-9, and use cross-validation of 7 folds.  For the best polynomial model in terms of $R^2$, does it outperform the baseline and the linear regression model in terms of $R^2$ and root mean square error?

In [38]:
pipe = Pipeline([
    ("poly", PolynomialFeatures(include_bias=False)),
    ("scaler", StandardScaler()),
    ("model", LinearRegression())
])

In [39]:
# polynomial degrees 0–9
param_grid = {"poly__degree": range(0, 10)}

grid = GridSearchCV(
    pipe,
    param_grid,
    cv=7,
    scoring="r2",
    n_jobs=-1
)

grid.fit(X_train, y_train)


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:516: FitFailedWarning: 
7 fits failed out of a total of 70.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
7 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/base.py", line 1365, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/Library/Frameworks/Python.framework/Versions/3.10/l

,estimator,Pipeline(step...egression())])
,param_grid,"{'poly__degree': range(0, 10)}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,7
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,degree,2


In [40]:
# find best model and degree
best_degree = grid.best_params_["poly__degree"]
best_model = grid.best_estimator_

print(f"Best polynomial degree: {best_degree}")
print(f"Best cross validation R^2: {grid.best_score_:.3f}")


Best polynomial degree: 2
Best cross validation R^2: 0.997


In [41]:
# performance eval
y_train_pred = best_model.predict(X_train)
y_test_pred = best_model.predict(X_test)

r2_train_poly = r2_score(y_train, y_train_pred)
r2_test_poly = r2_score(y_test, y_test_pred)
rmse_train_poly = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test_poly = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"R^2 (Train): {r2_train_poly:.3f}")
print(f"R^2 (Test): {r2_test_poly:.3f}")
print(f"RMSE (Train): {rmse_train_poly:.3f}")
print(f"RMSE (Test): {rmse_test_poly:.3f}")

R^2 (Train): 1.000
R^2 (Test): 1.000
RMSE (Train): 0.167
RMSE (Test): 0.184


In [42]:
# compare to baseline 
R_baseline_train = (10 ** (X_train["Zh"] / 10) / 200) ** (1 / 1.6)
R_baseline_test = (10 ** (X_test["Zh"] / 10) / 200) ** (1 / 1.6)

r2_train_base = r2_score(y_train, R_baseline_train)
r2_test_base = r2_score(y_test, R_baseline_test)
rmse_train_base = np.sqrt(mean_squared_error(y_train, R_baseline_train))
rmse_test_base = np.sqrt(mean_squared_error(y_test, R_baseline_test))

print(f"R^2 (Train): {r2_train_base:.3f}")
print(f"R^2 (Test): {r2_test_base:.3f}")
print(f"RMSE (Train): {rmse_train_base:.3f}")
print(f"RMSE (Test): {rmse_test_base:.3f}")

R^2 (Train): 0.276
R^2 (Test): 0.357
RMSE (Train): 7.144
RMSE (Test): 7.189


The model outperforms the linear regression model and baseline model in both R^2 and RMSE.

4. Repeat 1 with a Random Forest Regressor, and perform a grid_search on the following parameters:
   
   ```python
   param_grid = {
    "bootstrap": [True, False],
    "max_depth": [10, 100],
    "max_features": ["sqrt", 1.0],  
    "min_samples_leaf": [1, 4],
    "min_samples_split": [2, 10],
    "n_estimators": [200, 1000]}
   ```
  Can you beat the baseline, or the linear regression, or best polynomial model with the best optimized Random Forest Regressor in terms of $R^2$ and root mean square error?


In [43]:
# define  model
rf = RandomForestRegressor(random_state=42)

In [44]:
# parameter grid
param_grid = {
    "bootstrap": [True, False],
    "max_depth": [10, 100],
    "max_features": ["sqrt", 1.0],
    "min_samples_leaf": [1, 4],
    "min_samples_split": [2, 10],
    "n_estimators": [200, 1000],
}

In [45]:
# Grid search 
grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,                
    scoring="r2",
    n_jobs=-1,
    verbose=1
)

grid_search.fit(X_train, y_train)

print("Best RF params:", grid_search.best_params_)
print("Best CV R^2:", grid_search.best_score_)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best RF params: {'bootstrap': True, 'max_depth': 100, 'max_features': 1.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 1000}
Best CV R^2: 0.9795612504438205


In [46]:
# find best model
best_rf = grid_search.best_estimator_

In [47]:
y_train_pred = best_rf.predict(X_train)
y_test_pred = best_rf.predict(X_test)

In [48]:
# performance eval
r2_train_rf = r2_score(y_train, y_train_pred)
r2_test_rf = r2_score(y_test, y_test_pred)
rmse_train_rf = np.sqrt(mean_squared_error(y_train, y_train_pred))
rmse_test_rf = np.sqrt(mean_squared_error(y_test, y_test_pred))

print(f"R^2 (Train): {r2_train_rf:.3f}")
print(f"R^2 (Test):  {r2_test_rf:.3f}")
print(f"RMSE (Train): {rmse_train_rf:.3f}")
print(f"RMSE (Test):  {rmse_test_rf:.3f}")


R^2 (Train): 0.997
R^2 (Test):  0.988
RMSE (Train): 0.428
RMSE (Test):  0.978


The polynomial 2 model still performs best for both R^2 and RMSE.